In [1]:
import requests
import lxml.html as lh
import bs4 as bs
import urllib.request
import numpy as np 
import pandas as pd

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(url)
soup = bs.BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
data = pd.read_json(df[0].to_json(orient='records'))


In [3]:
data.head()

,Borough,Neighborhood,Postal Code
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,"Regent Park, Harbourfront",M5A


In [4]:
raw_data_selected = data[data['Borough'] != 'Not assigned']

In [ ]:
raw_data_selected = raw_data_selected.groupby(['Borough', 'Postal code'], as_index=False).agg(','.join)

In [17]:

raw_data_selected.head()

,Borough,Neighborhood,Postal Code
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,"Regent Park, Harbourfront",M5A
5,North York,"Lawrence Manor, Lawrence Heights",M6A
6,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M7A


In [19]:
raw_data_selected['Neighborhood'] = np.where(raw_data_selected['Neighborhood'] == 'Not assigned', raw_data_selected['Borough'], raw_data_selected['Neighborhood'])

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [20]:
raw_data_selected.shape

(103, 3)

# use of Geocoder or csv to create dataframe

In [9]:
geospatial_url = "https://cocl.us/Geospatial_data"
geospatial_data = pd.read_csv(geospatial_url)

In [10]:
geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
geospatial_data.columns = ['Postal Code', 'Latitude', 'Longitude']

In [22]:
geospatial_data.columns

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')

In [23]:
merged_data = pd.merge(raw_data_selected, geospatial_data, on='Postal Code')

In [24]:
merged_data.head()

,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,North York,Parkwoods,M3A,43.753259,-79.329656
1,North York,Victoria Village,M4A,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M7A,43.662301,-79.389494
